### Make datasets to analyze

Each dataset should have a training set `train.pkl`, a validation set `val.pkl`. If there is noise involved, there should be an additional `train_noiselss.pkl`, `dev_noiseless.pkl`.

This notebook serves as a manifest for reproducing precisely the datasets in this directory. Data access notes:
 - The data directory name is how you will access this data using `dataloader.py` 

In [1]:
# autoreload magic
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import os 
from mindreadingautobots.sequence_generators import make_datasets, data_io

### Generating newer datasets

In [6]:
# Generate data with bitflip values
n_val = 10000 # number of validation examples
seed = 1234
n_train = 2000
n_bits = 21 # number of TOTAL bits
# variables
p_bitflips = [0.05, 0.15, 0.4, 0.45]
# p_bitflips = [0.2]

def hamilton_6_choose_4(n_data, n_bits, p_bitflip, seed, **kwargs):
    return make_datasets.k_choose_m_hamilton_forecast_dataset(k=6, m=4, n_data=n_data, n_bits=n_bits, p_bitflip=p_bitflip, seed=seed, **kwargs)

def sparse_majority_k5(n_data, n_bits, p_bitflip, seed, subseq_idx=None):
    return make_datasets.sparse_majority_k_n(n_bits, 5, n_data, p_bitflip, seed, subseq_idx=subseq_idx)

def sparity_k4(n_data, n_bits, p_bitflip, seed, subseq_idx=None):
    """Wrapper for sparse_parity_k_n with k=4"""
    return make_datasets.sparse_parity_k_n(n_bits, 4, n_data, p_bitflip, seed, subseq_idx=subseq_idx)


generators = {
    # "parity_4lookback": make_datasets.parity_4lookback,
    # "not_majority_4lookback": make_datasets.not_majority_4lookback,
    # "sparse_parity_k4": sparity_k4,
    # "hamilton_6_choose_4": hamilton_6_choose_4,
    "sparse_majority_k5": sparse_majority_k5

}


for gen_name, generator in generators.items():
    # Use a fixed subseq_idx for all data from the same generator
    # This is useful for consistency across different p_bitflip values
    subseq_idx = None
    if gen_name == "hamilton_6_choose_4":
        subseq_idx = [2, 3, 4, 5]

    for p_bitflip in p_bitflips:
        p100 = int(p_bitflip*100)
        suffix = f"_nbits{n_bits}_n{n_train}_bf{p100}_seed{seed}"
        dirname = gen_name + suffix
        print(f"Generating {dirname} with p_bitflip={p_bitflip}")
        # If your dataset has a hidden subset, update this list:
        idx = None
        if subseq_idx is not None:
            idx = subseq_idx
        X, Z, subseq_idx = generator(n_train + n_val, n_bits, p_bitflip, seed, subseq_idx=idx) # now we keep the subseq_idx
        print("idx for sparse function: save these:", subseq_idx)

        if p_bitflip == 0:
            Z = X
        Z_train = Z[:n_train]
        Z_val = Z[n_train:]

        # Check if the data directory exists, if not create it
        if not os.path.exists(dirname):
            os.makedirs(dirname)

        train_path = f"{dirname}/train.pkl"
        val_path = f"{dirname}/val.pkl"
        data_io.save_numpy_as_dict(Z_train, train_path)
        data_io.save_numpy_as_dict(Z_val, val_path)

        X_train = X[:n_train]
        X_val = X[n_train:]
        noiseless_train_path = f"{dirname}/noiseless_train.pkl"
        noiseless_val_path = f"{dirname}/noiseless_val.pkl"
        data_io.save_numpy_as_dict(X_train, noiseless_train_path)
        data_io.save_numpy_as_dict(X_val, noiseless_val_path)


Generating sparse_majority_k5_nbits21_n2000_bf5_seed1234 with p_bitflip=0.05
idx for sparse function: save these: [ 3 13  2 16 14]
Generating sparse_majority_k5_nbits21_n2000_bf15_seed1234 with p_bitflip=0.15
idx for sparse function: save these: [ 3 13  2 16 14]
Generating sparse_majority_k5_nbits21_n2000_bf40_seed1234 with p_bitflip=0.4
idx for sparse function: save these: [ 3 13  2 16 14]
Generating sparse_majority_k5_nbits21_n2000_bf45_seed1234 with p_bitflip=0.45
idx for sparse function: save these: [ 3 13  2 16 14]


In [ ]:
print(X[0])

[1 1 0 1 0 0 0 1 1 1 1 1 0 0 1 0 0 0 0 0 0]
2


In [5]:
# Generate data with bitflip values
n_val = 10000 # number of validation examples 
seed = 1234 
n_train = 5000 # number of training examples
n_bits = 21 # number of TOTAL bits (including final bit)

# Create a different dataset for every 'nondeterministic' value in this list
# Note that _sometimes_ this means bitflip rate, but not always
nondeterms = [0.0, 0.1, 0.2]

generators = {
    # "parity_4lookback": make_datasets.parity_4lookback_nondeterministic,
    # "not_majority_4lookback": make_datasets.not_majority_4lookback_nondeterministic,
}

for nondeterm in nondeterms:
    p100 = int(nondeterm*100)
    suffix = f"_n{n_train}_nondeterm{p100}_seed{seed}"

    for gen_name, generator in generators.items():
        dirname = gen_name + suffix
        print(f"Generating {dirname} with nondeterm={nondeterm}")
        X, Z, idx = generator(n_train + n_val, n_bits, nondeterm, seed)
        print("PAY ATTENTION: idx=", idx)
        X_train = X[:n_train]
        X_val = X[n_train:]

        # Check if the data directory exists, if not create it
        if not os.path.exists(dirname):
            os.makedirs(dirname)

        train_path = f"{dirname}/train.pkl"
        val_path = f"{dirname}/val.pkl"
        data_io.save_numpy_as_dict(X_train, train_path)
        data_io.save_numpy_as_dict(X_val, val_path)

Generating parity_4lookback_n5000_nondeterm0_seed1234 with nondeterm=0.0


NotImplementedError: output signature is wrong, and seed bits are missing.

In [12]:
# load "./sparity40_1k/train.pkl" 
# dir_names = ['./sparity40_1k', 'sparity40_5k', 'sparity40_25h', 'sparse_n_parity4a', 'sparse_parity4a']
dir_names = ['sparse_parity4a']

for dir_name in dir_names:
    train_path = f'./{dir_name}/train.pkl'
    test_path = f'./{dir_name}/val.pkl'

    print(dir_name)
    with open(train_path, 'rb') as f:
        train = pickle.load(f)
    xvals = train.get('line')
    n_data = len(xvals)
    n_bits = len(xvals[0])
    print("train:", n_data, n_bits)

    # with open(test_path, 'rb') as f:
    #     test = pickle.load(f)
    # xvals = test.get('line')
    # n_data = len(xvals)
    # n_bits = len(xvals[0])
    # print("val:", n_data, n_bits)
    # print()

sparse_parity4a
train: 30000 40


In [21]:
	# test_path = os.path.join(data_path, config.dataset, 'test.tsv')
len('0010011001001111011011111011110111100010')

40

In [ ]:
voc= Voc()
train_path = './sparity40_1k/train.pkl'
z = voc.create_vocab_dict(None, path= train_path, debug = None)
voc.w2id 
